# Redis as a Context Store with OpenAI Chat
This notebook demonstrates how to use Redis as high-speed context memory with ChatGPT.

## Prerequisites
* Redis instance with the Redis Search and Redis JSON modules
* Redis-py client lib
* OpenAI Python client lib
* OpenAI API key

## Installation
Install Python modules necessary for the examples.

In [ ]:
! pip install -r requirements.txt

## OpenAI API Key
Create a .env file and add your OpenAI key to it

In [ ]:
OPENAI_API_KEY=your_key

## OpenAI Setup
Key load + helper function for chat completion

In [6]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

## Experiment - Chat Completion on a Topic outside of the Model's Knowledge Cutoff Date
Gpt-3.5-turbo was trained on data up to Sep 2021.  Let's ask it a question about something that is beyond that date.  In this case, the FTX/Sam Bankman-Fried scandal.

In [7]:
prompt = "Is Sam Bankman-Fried's company, FTX, considered a well-managed company?"
response = get_completion(prompt)
print(response)

As an AI language model, I cannot provide a personal opinion. However, FTX has been recognized as one of the fastest-growing cryptocurrency exchanges and has received positive reviews for its user-friendly interface, low fees, and innovative products. Additionally, Sam Bankman-Fried has been praised for his leadership and strategic decision-making, including FTX's recent acquisition of Blockfolio. Overall, FTX appears to be a well-managed company.


## Hallucinations
An unfortunate behavior of these AI systems is the system will provide a confident-sounding response - even when the system is not confident with its result.  This degenerative behavior has been termed 'hallucination'.  One way to mitigate hallucinations is prompt re-engineering, as seen below.

In [8]:
prompt ="Is Sam Bankman-Fried's company, FTX, considered a well-managed company?  If you don't know for certain, say unknown."
response = get_completion(prompt)
print(response)

Unknown.


## Additional Context
Another way to combat hallucinations is to give the system more information such that it can make intelligent decisions vs guessing.  We'll use Redis as the source for that additional context.  We'll pull in business news articles from after the GPT knowledge cut-off date such that the system will have a better understanding of how FTX was actually managed.

To complete the 'hallucination' metaphor, we can say Redis is acting as an anti-psychotic medication.  

## Start the Redis Stack Docker container

In [11]:
! docker compose up -d

[+] Running 0/0
 ⠋ Network redisqna_default  Creating                                      0.1s 
[+] Running 1/1
 ✔ Network redisqna_default    Created                                     0.1s 
 ⠿ Container redisqna-redis-1  Starting                                    0.1s 
[+] Running 1/2
 ✔ Network redisqna_default    Created                                     0.1s 
 ⠿ Container redisqna-redis-1  Starting                                    0.2s 
[+] Running 1/2
 ✔ Network redisqna_default    Created                                     0.1s 
 ⠿ Container redisqna-redis-1  Starting                                    0.3s 
[+] Running 2/2
 ✔ Network redisqna_default    Created                                     0.1s 
 ✔ Container redisqna-redis-1  Started                                     0.4s 


## Connect Redis client

In [12]:
from redis import from_url

REDIS_URL = 'redis://localhost:6379'
client = from_url(REDIS_URL)
client.ping()

True

## Create Index

In [13]:
from redis.commands.search.field import TextField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType

schema = [ VectorField('$.vector', 
            "FLAT", 
            {   "TYPE": 'FLOAT32', 
                "DIM": 1536, 
                "DISTANCE_METRIC": "COSINE"
            },  as_name='vector' ),
            TextField('$.content', as_name='content')
        ]
idx_def = IndexDefinition(index_type=IndexType.JSON, prefix=['doc:'])
client.ft('idx').create_index(schema, definition=idx_def)

b'OK'

## Load Data Files into Redis as JSON Objects with Text and Vector Fields

In [14]:
import os
import openai

directory = './assets/'
model='text-embedding-ada-002'
i = 1
for file in os.listdir(directory):
    with open(os.path.join(directory, file)) as f:
        content = f.read()
        vector = openai.Embedding.create(input = [content], model = model)['data'][0]['embedding']
        client.json().set(f'doc:{i}', '$', {'content': content, 'vector': vector})
    i += 1

## Embed the Question and Perform VSS to find the most relevant document

In [15]:
from redis.commands.search.query import Query
import numpy as np

vec = np.array(openai.Embedding.create(input = [prompt], model = model)['data'][0]['embedding'], dtype=np.float32).tobytes()
q = Query('*=>[KNN 1 @vector $query_vec AS vector_score]')\
    .sort_by('vector_score')\
    .return_fields('content')\
    .dialect(2)    
params = {"query_vec": vec}

context = client.ft('idx').search(q, query_params=params).docs[0].content

## Repeat the Question to OpenAI with context
Now that we have relevant context, add that to the prompt to OpenAI and get a very different response.

In [16]:
prompt = f"""
Using the information delimited by triple backticks, answer this question: Is Sam Bankman-Fried's company, FTX, considered a well-managed company?

Context: ```{context}```
"""

response = get_completion(prompt)
print(response)

No, Sam Bankman-Fried's company FTX is not considered a well-managed company as it has filed for bankruptcy and owes as much as $8 billion to its creditors. The collapse of FTX has destabilized the crypto industry, and the company is already the target of investigations by the Securities and Exchange Commission and the Justice Department. FTX was widely viewed as one of the most stable and responsible companies in the freewheeling, loosely regulated crypto industry, but its risky practices have become pervasive in crypto, leading to a reckoning.
